In [345]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%inline matplotlib


UsageError: Line magic function `%inline` not found.


In [346]:
#We used a (bad) model to generate a large list of names. now we will try to build a classifier to see if we can tell the difference between real and fake names.
real_names = open('names.txt', 'r').read().splitlines()
fake_names = open('generated_names.txt', 'r').read().splitlines()
len(real_names), len(fake_names)
#see if theere are names in both lists
for name in real_names:
    if name in fake_names:
        print(name)
        break


In [347]:
chars = sorted(list(set(''.join(real_names+fake_names))))
stoi = {c:i+1 for i,c in enumerate(chars)}
stoi['.'] = 0
itos = {i:c for c,i in stoi.items()}


In [348]:
#find longest name
maxlen = max([len(n) for n in real_names+fake_names])
maxlen

#pad all names to maxlen
real_names = [n.ljust(maxlen, '.') for n in real_names]
fake_names = [n.ljust(maxlen, '.') for n in fake_names]

#print first 10 names
real_names[:10]




['emma................',
 'olivia..............',
 'ava.................',
 'isabella............',
 'sophia..............',
 'charlotte...........',
 'mia.................',
 'amelia..............',
 'harper..............',
 'evelyn..............']

In [349]:
# create Y's
Y = torch.cat([torch.ones(len(real_names)), torch.zeros(len(fake_names))])
Y.shape
#add a dimension to Y
Y = Y.unsqueeze(1)


# create X's
X = torch.cat([torch.LongTensor([[stoi[c] for c in n]]) for n in real_names+fake_names])
X.shape





torch.Size([64066, 20])

In [350]:
embedding_dim = 12

W1 = torch.randn(X.shape[1]*embedding_dim, 750, requires_grad=True)
b1 = torch.randn(750, requires_grad=True)
W2 = torch.randn(750, 1, requires_grad=True)
b2 = torch.randn(1, requires_grad=True)

C = torch.randn((27, embedding_dim), requires_grad=True)
params = [W1, b1, W2, b2, C] 


def accuracy(pred, Y):
    return (pred.round() == Y).float().mean()


lr = 0.01




In [351]:
batch_size = 64
for i in range(100000):
    ix = torch.randint(0, X.shape[0], (batch_size,))
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 20*embedding_dim) @ W1 + b1)
    logits = h @ W2 + b2
    #we use binary cross entropy loss because we are doing binary classification
    loss = F.binary_cross_entropy_with_logits(logits, Y[ix])
    loss.backward()
    acc = accuracy(logits, Y[ix])
    for p in params:
        p.data -= p.grad * lr
        p.grad.zero_()
    if i % 100 == 0:
        print(f'loss: {loss.data:.3f}')

loss: 5.199
loss: 1.587
loss: 0.847
loss: 0.093
loss: 0.098
loss: 0.029
loss: 0.005
loss: 0.323
loss: 0.034
loss: 0.035
loss: 0.366
loss: 0.032
loss: 0.374
loss: 0.002
loss: 0.699
loss: 0.187
loss: 0.061
loss: 0.001
loss: 0.619
loss: 0.003
loss: 0.013
loss: 0.004
loss: 0.000
loss: 0.010
loss: 0.003
loss: 0.000
loss: 0.000
loss: 0.001
loss: 0.000
loss: 0.000
loss: 0.001
loss: 0.000
loss: 0.001
loss: 0.000
loss: 0.000
loss: 0.001
loss: 0.001
loss: 0.000
loss: 0.000
loss: 0.079
loss: 0.001
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.001
loss: 0.572
loss: 0.000
loss: 0.005
loss: 0.001
loss: 0.000
loss: 0.010
loss: 0.000
loss: 0.003
loss: 0.001
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.004
loss: 0.000
loss: 0.195
loss: 0.000
loss: 0.006
loss: 0.001
loss: 0.000
loss: 0.001
loss: 0.000
loss: 0.000
loss: 0.002
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.098
loss: 0.065
loss: 0.002
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.000
loss: 0.000
loss

KeyboardInterrupt: 

In [352]:
#accuracy on whole dataset
pred = torch.sigmoid((torch.tanh(C[X].view(-1, 20*embedding_dim) @ W1 + b1) @ W2 + b2))
accuracy(pred, Y)



tensor(1.)

In [354]:
#show examples of where we got it wrong
wrong = (pred.round() != Y).squeeze()
for i in torch.where(wrong)[0][:10]:
    print(real_names[i], pred[i].item())
    
